In [97]:
import numpy as np
from pandas import *
from preprocessing import getData

In [98]:
def mappingFunction(X, InputSize, HiddenNeurons, RandomA, RandomB, name):
    HiddenLayer = np.zeros((InputSize, HiddenNeurons))
    if name == "Gaussian":
        # H = exp(-b||x - a||)
        for i in range(InputSize):
            for j in range(HiddenNeurons):
                HiddenLayer[i][j] = np.exp(-RandomB[0][j] * np.linalg.norm(X[i] - RandomA.T[j], 1))
                if HiddenLayer[i][j] == 0:
                    HiddenLayer[i][j] = 1e-10
    elif name == "Tanh":
        for i in range(InputSize):
            for j in range(HiddenNeurons):
                ExponentValue = np.exp(-(X[i].dot(RandomA.T[j].T) + RandomB[0][j]))
                HiddenLayer[i][j] = (1 - ExponentValue)/(1 + ExponentValue)
    return HiddenLayer

In [99]:
def trainAndTest(train_X, train_Y, test_X, test_Y, name, HiddenNeurons):
    # TRAINING
    TrainSize = train_X.shape[0]
    InputNeurons = train_X.shape[1]
    # Random Feature Mapping, Evaluating Hidden Layer Matrix
    if name == "Gaussian":
        RandomA = np.random.rand(InputNeurons, HiddenNeurons)
        RandomB = np.random.rand(1, HiddenNeurons)
    else:
        RandomA = np.random.randn(InputNeurons, HiddenNeurons)
        RandomB = np.random.randn(1, HiddenNeurons)
    HiddenLayer = mappingFunction(train_X, TrainSize, HiddenNeurons, RandomA, RandomB, name)
    # Evaluating Weight Matrix
    #WeightMatrix = np.linalg.inv(HiddenLayer.T.dot(HiddenLayer)).dot(HiddenLayer.T).dot(train_Y)
#     WeightMatrix = np.linalg.pinv(HiddenLayer).dot(train_Y)
    WeightMatrix = np.linalg.inv(HiddenLayer.T.dot(HiddenLayer)).dot(HiddenLayer.T).dot(train_Y)
    
    # TESTING
    TestSize = test_X.shape[0]
    TestHiddenLayer = mappingFunction(test_X, TestSize, HiddenNeurons, RandomA, RandomB, name)
    PredictedOutput = TestHiddenLayer.dot(WeightMatrix)
    #Compare with actual output
    count = 0
    TrueZeros = TrueOnes = FalseZeros = FalseOnes = 0
    for i in range(TestSize):
        actualClass = np.argmax(test_Y[i])
        predictedClass = np.argmax(PredictedOutput[i])
        if actualClass == predictedClass:
            count += 1
            if actualClass == 0:
                TrueZeros += 1
            else:
                TrueOnes += 1
        else:
            if actualClass == 0:
                FalseZeros += 1
            else:
                FalseOnes += 1
    conf_mat = ([[TrueZeros, FalseZeros], [FalseOnes, TrueOnes]])
    return [count/TestSize*100, conf_mat]

In [104]:
#5 fold cross validation
from sklearn.model_selection import KFold
def k_fold(k, name):
    kf = KFold(n_splits = k)
    kf.get_n_splits(X)
    fold = 0
    avg_accuracy = 0
    print("{} function :- \n".format(name))
    for train_index, test_index in kf.split(X):
        fold += 1
        train_X, test_X = X[train_index], X[test_index]
        train_Y, test_Y = Y[train_index], Y[test_index]
        [accuracy, conf_mat] =  trainAndTest(train_X, train_Y, test_X, test_Y, name, HiddenNeurons)
        print("Confusion matrix :- ")
        print(DataFrame(conf_mat))
        print()
        print("Fold %d Accuracy :- %s"%(fold, accuracy),"%")
        avg_accuracy += accuracy
    print("Average Accuracy is :- %s"%(avg_accuracy/fold),"%")

In [105]:
X, Y = getData('data.mat')

In [106]:
HiddenNeurons = 350
print("Number of hidden neurons :- ",HiddenNeurons)
k_fold(5, "Gaussian")

Number of hidden neurons :-  350
Gaussian function :- 

Confusion matrix :- 
     0    1
0  175   34
1   32  189

Fold 1 Accuracy :- 84.65116279069768 %
Confusion matrix :- 
     0    1
0  159   56
1   41  174

Fold 2 Accuracy :- 77.44186046511628 %
Confusion matrix :- 
     0    1
0  175   32
1   60  163

Fold 3 Accuracy :- 78.6046511627907 %
Confusion matrix :- 
     0    1
0  170   52
1   41  166

Fold 4 Accuracy :- 78.32167832167832 %
Confusion matrix :- 
     0    1
0  171   51
1   43  164

Fold 5 Accuracy :- 78.08857808857809 %
Average Accuracy is :- 79.42158616577221 %


In [144]:
HiddenNeurons = 315
print("Number of hidden neurons :- ",HiddenNeurons)
k_fold(5, "Tanh")

Number of hidden neurons :-  315
Tanh function :- 

Confusion matrix :- 
     0    1
0  185   24
1   24  197

Fold 1 Accuracy :- 88.83720930232558 %
Confusion matrix :- 
     0    1
0  181   34
1   30  185

Fold 2 Accuracy :- 85.11627906976744 %
Confusion matrix :- 
     0    1
0  176   31
1   29  194

Fold 3 Accuracy :- 86.04651162790698 %
Confusion matrix :- 
     0    1
0  200   22
1   21  186

Fold 4 Accuracy :- 89.97668997668997 %
Confusion matrix :- 
     0    1
0  183   39
1   24  183

Fold 5 Accuracy :- 85.3146853146853 %
Average Accuracy is :- 87.05827505827506 %
